In [112]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import talib as ta
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [12]:
apple_data = pd.read_csv("./Datasets/Historical/HistoricalData_APPLE.csv", parse_dates=["Date"], index_col="Date")
apple_data.index = pd.to_datetime(apple_data.index,format='%Y-%m-%d')
apple_data = apple_data.sort_index(ascending=True, axis=0)
apple_data.head()

,Close/Last,Volume,Open,High,Low
Date,,,,,
2011-06-10,$11.6393,433801306,$11.8054,$11.845,$11.6254
2011-06-13,$11.6643,329376468,$11.6857,$11.7254,$11.6096
2011-06-14,$11.8729,333995906,$11.7857,$11.9018,$11.7611
2011-06-15,$11.6696,395841722,$11.7768,$11.7964,$11.6029
2011-06-16,$11.6129,507299317,$11.675,$11.7386,$11.3689


In [13]:
apple_data.shape

(2516, 5)

In [14]:
apple_data['Open'] = apple_data['Open'].apply(lambda x: x.replace('$',''))
apple_data['Open'] = apple_data['Open'].astype(float)

apple_data['Close/Last'] = apple_data['Close/Last'].apply(lambda x: x.replace('$',''))
apple_data['Close/Last'] = apple_data['Close/Last'].astype(float)

apple_data['High'] = apple_data['High'].apply(lambda x: x.replace('$',''))
apple_data['High'] = apple_data['High'].astype(float)

apple_data['Low'] = apple_data['Low'].apply(lambda x: x.replace('$',''))
apple_data['Low'] = apple_data['Low'].astype(float)

In [15]:
apple_data.describe()

,Close/Last,Volume,Open,High,Low
count,2516.000000,2.516000e+03,2516.000000,2516.000000,2516.000000
mean,41.277389,2.388826e+08,41.268871,41.698470,40.829849
std,30.304841,1.914594e+08,30.319067,30.698585,29.893474
min,11.261400,4.544820e+07,11.310000,11.346400,11.089300
25%,21.215550,1.092658e+08,21.244100,21.423600,20.992800
50%,29.592500,1.679334e+08,29.505000,29.810000,29.281250
75%,47.733750,3.022477e+08,47.770625,47.996250,47.371250
max,143.160000,1.498071e+09,143.600000,145.090000,141.370000


In [16]:
apple_data['S_10'] = apple_data['Close/Last'].rolling(window=10).mean()
apple_data['Corr'] = apple_data['Close/Last'].rolling(window=10).corr(apple_data['S_10'])
apple_data['RSI'] = ta.RSI(np.array(apple_data['Close/Last']), timeperiod =10)
apple_data['Open-Close'] = apple_data['Open'] - apple_data['Close/Last'].shift(1)
apple_data['Open-Open'] = apple_data['Open'] - apple_data['Open'].shift(1)
apple_data = apple_data.dropna()
X = apple_data.iloc[:,:9]

In [17]:
X

,Close/Last,Volume,Open,High,Low,S_10,Corr,RSI,Open-Close
Date,,,,,,,,,
2011-07-07,12.7571,398385801,12.6668,12.7857,12.6429,12.12943,0.975606,75.187594,0.1039
2011-07-08,12.8468,489331164,12.6193,12.8571,12.5786,12.23115,0.975247,76.614376,-0.1378
2011-07-11,12.6429,441465743,12.7264,12.8489,12.6007,12.32990,0.938690,66.898395,-0.1204
2011-07-12,12.6339,450866740,12.6261,12.7743,12.4507,12.40743,0.890211,66.484888,-0.0168
2011-07-13,12.7864,390229404,12.7975,12.8571,12.7279,12.48871,0.869988,69.978565,0.1636
...,...,...,...,...,...,...,...,...,...
2021-06-03,123.5400,76229170,124.6800,124.8500,123.1300,125.63600,0.372929,38.038837,-0.3800
2021-06-04,125.8900,75169340,124.0700,126.1600,123.8500,125.49400,0.155174,48.941592,0.5300
2021-06-07,125.9000,71057550,126.1700,126.3200,124.8321,125.54100,0.106117,48.984035,0.2800


In [50]:
Y = np.where(apple_data['Close/Last'].shift(-1) > apple_data['Close/Last'],1,0)
Y

array([1, 0, 0, ..., 1, 1, 0])

In [78]:
split = int(0.7*len(apple_data))
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [79]:
log_model = LogisticRegression()
log_model = log_model.fit(X_train,Y_train)

In [80]:
pd.DataFrame(zip(X.columns, np.transpose(log_model.coef_)))

,0,1
0,Close/Last,[2.1353967115329352e-17]
1,Volume,[9.765555004474918e-11]
2,Open,[2.1663012260688275e-17]
3,High,[2.1731279882691568e-17]
4,Low,[2.1370004857113262e-17]
5,S_10,[2.0692004121819012e-17]
6,Corr,[1.0343380419403885e-18]
7,RSI,[7.919555985055324e-17]
8,Open-Close,[1.0027875778830853e-19]


In [81]:
probability = log_model.predict_proba(X_test)
probability

array([[0.49819744, 0.50180256],
       [0.49672482, 0.50327518],
       [0.49799904, 0.50200096],
       ...,
       [0.49826522, 0.50173478],
       [0.49818352, 0.50181648],
       [0.49861139, 0.50138861]])

In [82]:
Y_predicted = log_model.predict(X_test)

In [83]:
print(metrics.confusion_matrix(Y_test,Y_predicted))

[[  0 343]
 [  0 407]]


In [84]:
log_model.score(X_test,Y_test)

0.5426666666666666

In [85]:
print(metrics.classification_report(Y_test, Y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       343
           1       0.54      1.00      0.70       407

    accuracy                           0.54       750
   macro avg       0.27      0.50      0.35       750
weighted avg       0.29      0.54      0.38       750

C:\Users\sidha\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sidha\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

In [90]:
from sklearn.model_selection import GridSearchCV

c_space = np.logspace(5, 20, 70)
param_grid = {'C': c_space}

logreg = LogisticRegression()

logreg_cv = GridSearchCV(logreg, param_grid, cv = 6)
  
logreg_cv.fit(X, Y)

print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 100000.0}
Best score is 0.5240192461415484
